In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import mmap
import random
import pickle
import tiktoken
import regex as re

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#model name model-<model number>_<0 + saved model usedd>_<iter>
model_name = "V:/llm-project/trained_models/model-01_1_subWord.pkl"
saved_model = 'V:/llm-project/trained_models/model-03_5.pkl'
use_trained_model = False

train_file_path = "V:/llm-project/datasets/sw_train_split.txt"
val_file_path = "V:/llm-project/datasets/sw_val_split.txt"
vocab_file_path = "V:/llm-project/datasets/sw_vocab.txt"

batch_size = 32
block_size = 64
learning_rate = 3e-4
max_iters = 500
eval_iters = 50
n_embd = 192 #384
n_head = 2
n_layer = 2
dropout = 0.2

cuda


In [62]:
def Convert(string): 
    li = list(string.split('\n')) 
    return li

with open(vocab_file_path, 'r', encoding='utf-8') as f:
    text = f.read()

print(f'text: {text[:100]}')
sub_word = Convert(text)
print(f'txt: {sub_word[:10]}')
vocab_size = len(sub_word)
print(vocab_size)

text:  Withstood 
 magnetosphere 
51663 
0117119 
0118655 
 ▸ 
13077 
 Chichewa 
fscanf 
Azmy 
8057993 
00
txt: [' Withstood ', ' magnetosphere ', '51663 ', '0117119 ', '0118655 ', ' ▸ ', '13077 ', ' Chichewa ', 'fscanf ', 'Azmy ']
1755846


In [63]:
#word level encoding
string_to_int = {sb:i for i,sb in enumerate(sub_word)}
int_to_string = {i:sb for i,sb in enumerate(sub_word)}

encode = lambda s: [string_to_int[sub_word] for sub_word in s.split()]
decode = lambda l: ' '.join([int_to_string[i] for i in l])

# GPT4 encoding/tokenizer?
enc = tiktoken.get_encoding('cl100k_base')

# test = 'this is random text'
# print(f'test: {encode(test)}')
# print(f'test: {decode(encode(test))}')
# print(data[:100])
# print([f'shape: {data.shape}'])

In [64]:
def get_random_chunk(split):
    filename = train_file_path if split == 'train' else val_file_path
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            # decoded_block = decode(block)

            # regex = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
            # rgx = re.findall(regex, decoded_block) #regex get words and break things like 12d124d1d4g3g
            # text = "" . join(rgx)
            # print(f'b4 tensor: {text}')

            # Train and test splits
            data = torch.tensor(enc.encode(decoded_block), dtype=torch.long)
            
    print(f'after tensor: {data}')
    return data

def get_batch(split):
    # data = train_data if split == 'train' else val_data
    data = get_random_chunk(split)
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i + 1:i+block_size + 1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

after tensor: tensor([   16,   188,   220,    15,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   592,   277,   220,   220,   188,   188,
          188,   188,   188,   188,   188,   188,   188,   188,   188,   188,
          188,   188,   188,   188,   188,   188, 

In [65]:
@torch.no_grad
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [66]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)

if(use_trained_model):
    print('loading model parameters...')
    with open(saved_model, 'rb') as f:
        model = pickle.load(f)
    print('loaded successfully!')


m = model.to(device)

In [67]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, eval loss: {losses["val"]:.4f}')
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

with open(model_name, "wb") as f:
    pickle.dump(model, f)
print("model saved")

after tensor: tensor([  188,   188,   188,  ..., 18808,   369, 16163])


OutOfMemoryError: CUDA out of memory. Tried to allocate 13.40 GiB. GPU 

In [ ]:
# prompt = input("Prompt:\n")
prompt = 'hello'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=150)[0].tolist())
print(f'Completion:\n{generated_chars}')

Completion:
hellogg ssaidactedvern playe the vreass and to pirneds whoto therem.smatar gast refing bedur Jaysload pornets coTm what to catter balsors.



In but-ith is


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=150)[0].tolist())
print(generated_chars)

                                                                                                                                                       
